In [4]:
import os
from app_decomposer.utils_demo import *
from pprint import pprint

ModuleNotFoundError: No module named 'simpy'

In [ ]:
ts = get_job_timeseries_from_file()
#ts = ts.pop("accessPattern")
pprint(ts["volume"])
#ts = ts.pop('processCount')

In [ ]:
job1 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607510, 1581607515, 1581607520, 1581607525])}

job2 = {'bytesRead': np.array([       2457,           8000,  0,  0]),
        'bytesWritten': np.array([         15404800,  91795200,   0,          0]),
        'timestamp': np.array([1581607520, 1581607525, 1581607530, 1581607535])}

job3 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607530, 1581607535, 1581607540, 1581607545])}

In [ ]:

jobid=3918
dataset_path = os.path.join(dirname(dirname(os.getcwd())), "dataset_generation", "dataset_generation")
job_files, job_ids, dataset_names = list_jobs(dataset_path)
# create two lists of job_ids where only few elements are differents


print(dataset_path)
print(job_ids)
print(job_files)
csv_file = job_files[job_ids.index("52508")]
df = pd.read_csv(csv_file, index_col=0)  # read the csv file into a dataframe
# print the columns of the dataframe to check if 'operationRead' exists
print(csv_file)
get_job_timeseries_from_file(job_id=7910)


In [2]:
import sys
sys.path.append("../")
import os
import unittest
from shutil import rmtree
import urllib3
from loguru import logger
from pydantic import BaseModel
from typing import List
from app_decomposer.connector import Connector, APIConnector, ApiModel
from app_decomposer.config_parser import Configuration
import unittest
import numpy as np
import pandas as pd
from unittest.mock import MagicMock
from app_decomposer.central_job import WorkflowSynthesizer, CentralJob, WorkflowSearcher,display_features_3d, display_timeseries

CONFIG_FILE = os.path.join(os.path.dirname(os.getcwd()), "tests_integration",
                      "test_data", "test_docker_config.yaml")

In [3]:
config = Configuration(path=CONFIG_FILE)
keycloak_token = config.get_kc_token_docker()
connector = APIConnector(api_uri="http://localhost/pybackend/",
                              api_token=f"Bearer {keycloak_token}")

connector.check_connection()

False

In [34]:
searcher = WorkflowSearcher(connector)
df = searcher.search_workflows("Cryo_EM")
print(df.shape)
print(df['id'])
df.head()

2023-07-18 10:45:00.709 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending POST request to endpoint /ioi/workflows/


(13, 10)
0     642b5fd8547fb888bac488ed
1     633ab5ebac213a8b8529d186
2     634019e6ac213a8b852f5f9c
3     63402075ac213a8b852f6821
4     634015a2ac213a8b852f4ec5
5     63e52bfdac213a8b85e98c21
6     63be91efac213a8b85bc56eb
7     63be952fac213a8b85bc65c8
8     63455953ac213a8b853535f4
9     63bfc87bac213a8b85be494b
10    643776f0547fb888bad22e64
11    634d3df5ac213a8b853d659e
12    633313fcac213a8b852232db
Name: id, dtype: object


,id,instanceNumber,name,username,version,jobs,workflowStartTime,workflowEndTime,workflowDuration,accumulatedJobDuration
0,642b5fd8547fb888bac488ed,03042023,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 343407}],1680564180,1680564780,600,600
1,633ab5ebac213a8b8529d186,03102022,Cryo_EM,furmanek1,4.0.5-Bull.9,"[{'jobid': 273857}, {'jobid': 273858}, {'jobid...",1664792040,1665143626,351586,1520
2,634019e6ac213a8b852f5f9c,07102022_02,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277150}],1665145315,1665145367,52,52
3,63402075ac213a8b852f6821,07102022_large,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277155}],1665146995,1665150606,3611,3611
4,634015a2ac213a8b852f4ec5,071022_01,Cryo_EM,furmanek1,4.0.5-Bull.9,[{'jobid': 277142}],1665144225,1665144787,562,562


In [35]:
# Extract workflows data
workflows = [searcher.extract_workflow_data(workflow_id) for workflow_id in df['id']]

#print(workflows)


2023-07-18 10:45:01.974 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/642b5fd8547fb888bac488ed
2023-07-18 10:45:02.239 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/633ab5ebac213a8b8529d186
2023-07-18 10:45:02.651 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/634019e6ac213a8b852f5f9c
2023-07-18 10:45:02.892 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/63402075ac213a8b852f6821
2023-07-18 10:45:03.214 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/634015a2ac213a8b852f4ec5
2023-07-18 10:45:03.486 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/63e52bfdac213a8b85e98c21
2023-07-18 10:45

In [36]:
# Convert each workflow data into the format expected by CentralJob
central_jobs = {}
for workflow in workflows:
    for workflow_id, workflow_data in workflow.items():
        job = {
            'bytesRead': np.array(workflow_data['bytesRead']),
            'bytesWritten': np.array(workflow_data['bytesWritten'])
        }
        central_jobs[workflow_id] = job

# Use the CentralJob class to find the central job id for all workflows
central_job = CentralJob(central_jobs)
central_job_id = central_job.find_central_job()

# Print the id of the central job
print('Central job ID:', central_job_id)

# Extract corresponding workflow using its id
central_workflow_data = central_jobs[central_job_id]
print('Central workflow ID:', central_job_id)
#print('Central workflow data:', central_workflow_data)



2023-07-18 10:45:07.243 | INFO     | app_decomposer.central_job:__init__:141 - Initializing CentralJob instance
2023-07-18 10:45:07.245 | INFO     | app_decomposer.central_job:find_central_job:220 - Finding central job
2023-07-18 10:45:07.246 | INFO     | app_decomposer.central_job:process:186 - Processing jobs data
2023-07-18 10:45:07.247 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.250 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.251 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.252 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.252 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.254 | INFO     | app_decomposer.central_job:fft_features:161 - Calculating FFT features
2023-07-18 10:45:07.254 | INFO    

Central job ID: 63be91efac213a8b85bc56eb
Central workflow ID: 63be91efac213a8b85bc56eb


In [37]:
central_job._features

,bytesRead_min,bytesRead_max,bytesRead_mean,bytesRead_dft_0,bytesRead_dft_1,bytesRead_dft_2,bytesRead_dft_3,bytesRead_dft_4,bytesRead_dft_5,bytesRead_dft_6,...,bytesWritten_dft_10,bytesWritten_dft_11,bytesWritten_dft_12,bytesWritten_dft_13,bytesWritten_dft_14,bytesWritten_dft_15,bytesWritten_dft_16,bytesWritten_dft_17,bytesWritten_dft_18,bytesWritten_dft_19
job_id,,,,,,,,,,,,,,,,,,,,,
642b5fd8547fb888bac488ed,0.0,0.001747,0.000690,0.001222,0.000918,0.000388,0.001419,0.002594,0.002731,0.001352,...,2.563065e-06,1.838491e-06,1.853780e-06,2.029051e-06,1.587461e-06,1.519814e-06,1.363934e-06,1.511118e-06,1.221836e-06,1.113118e-06
633ab5ebac213a8b8529d186,0.0,0.962097,0.154479,0.001187,0.001258,0.001507,0.002104,0.003024,0.004425,0.002954,...,9.225699e-09,9.018323e-09,8.487477e-09,7.820096e-09,7.167530e-09,6.605807e-09,6.158287e-09,5.823947e-09,5.594139e-09,5.460160e-09
634019e6ac213a8b852f5f9c,0.0,0.123374,1.000000,1.000000,1.000000,1.000000,1.000000,0.865567,1.000000,1.000000,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
63402075ac213a8b852f6821,0.0,0.048056,0.070443,0.070938,0.029323,0.019757,0.002569,0.005523,0.064801,0.017527,...,6.858663e-08,1.420522e-07,6.884471e-07,5.961813e-07,3.600512e-08,9.951889e-07,1.141671e-06,5.705185e-08,1.977049e-06,4.410599e-06
634015a2ac213a8b852f4ec5,0.0,0.095130,0.071046,0.071540,0.071035,0.069208,0.064546,0.044812,0.053547,0.069293,...,8.107637e-02,8.107654e-02,8.107689e-02,8.107718e-02,8.107739e-02,8.107760e-02,8.107793e-02,8.107840e-02,8.107891e-02,8.107930e-02
63e52bfdac213a8b85e98c21,0.0,0.000514,0.006995,0.007523,0.007741,0.009077,0.011666,0.015460,0.019404,0.010918,...,6.212592e-08,6.072945e-08,5.715473e-08,5.266058e-08,4.826620e-08,4.448355e-08,4.146995e-08,3.921850e-08,3.767097e-08,3.676875e-08
63be91efac213a8b85bc56eb,0.0,0.147016,0.072331,0.072825,0.069635,0.059722,0.048274,0.083983,0.200641,0.169692,...,1.136528e-01,1.114003e-01,1.056635e-01,9.852018e-02,9.162588e-02,8.578000e-02,8.119461e-02,7.781877e-02,7.552706e-02,7.420304e-02
63be952fac213a8b85bc65c8,0.0,0.310124,0.050755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
63455953ac213a8b853535f4,0.0,0.065778,0.060204,0.060704,0.025050,0.027536,0.009167,0.044763,0.048648,0.004101,...,2.583961e-07,2.131994e-07,9.252032e-07,8.341418e-07,4.492827e-07,7.596646e-07,1.543705e-06,3.653534e-07,2.159802e-06,4.297642e-06


In [38]:
features = central_job._features

print(features.columns)
print(features)

Index(['bytesRead_min', 'bytesRead_max', 'bytesRead_mean', 'bytesRead_dft_0',
       'bytesRead_dft_1', 'bytesRead_dft_2', 'bytesRead_dft_3',
       'bytesRead_dft_4', 'bytesRead_dft_5', 'bytesRead_dft_6',
       'bytesRead_dft_7', 'bytesRead_dft_8', 'bytesRead_dft_9',
       'bytesRead_dft_10', 'bytesRead_dft_11', 'bytesRead_dft_12',
       'bytesRead_dft_13', 'bytesRead_dft_14', 'bytesRead_dft_15',
       'bytesRead_dft_16', 'bytesRead_dft_17', 'bytesRead_dft_18',
       'bytesRead_dft_19', 'bytesWritten_min', 'bytesWritten_max',
       'bytesWritten_mean', 'bytesWritten_dft_0', 'bytesWritten_dft_1',
       'bytesWritten_dft_2', 'bytesWritten_dft_3', 'bytesWritten_dft_4',
       'bytesWritten_dft_5', 'bytesWritten_dft_6', 'bytesWritten_dft_7',
       'bytesWritten_dft_8', 'bytesWritten_dft_9', 'bytesWritten_dft_10',
       'bytesWritten_dft_11', 'bytesWritten_dft_12', 'bytesWritten_dft_13',
       'bytesWritten_dft_14', 'bytesWritten_dft_15', 'bytesWritten_dft_16',
       'bytesWritt

In [71]:
import plotly.graph_objs as go
from sklearn.decomposition import PCA
import plotly.subplots as sp
import math
from sklearn.manifold import TSNE
import umap
from sklearn.metrics.pairwise import euclidean_distances

def display_features(features, central_job_id, dim_reduction='PCA', seed=42, 
                     n_components=3, add_centroid=True):
    """
    Display the features using PCA, t-SNE and Plotly. The job closest to the centroid is highlighted.
    """
    
    def add_centroid_to_features(df, job_id="centroid"):
        """Adds a virtual job to the dataframe with features being the average of all other jobs."""
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        centroid_features = df[numeric_cols].mean(axis=0)
        centroid_df = pd.DataFrame([centroid_features], columns=df.columns)
        centroid_df['job_id'] = job_id
        return pd.concat([df, centroid_df])


    # Reset index
    features_df = features.reset_index()
    if add_centroid:
        features_df = add_centroid_to_features(features_df)
    # Separate features from job_id
    job_ids = features_df['job_id'].values
    features = features_df.drop(columns='job_id')

    # Perform dimensionality reduction
    if dim_reduction == 'PCA':
        reducer = PCA(n_components=n_components, random_state=seed)
    elif dim_reduction == 't-SNE':
        if features.shape[0] <= n_components:  # set perplexity value to be less than the number of samples
            perplexity = features.shape[0] - 1
        else:
            perplexity = n_components
        reducer = TSNE(n_components=n_components, random_state=seed, perplexity=perplexity)
    else:
        raise ValueError(f"Unknown dimensionality reduction method: {dim_reduction}")

    reduced_features = reducer.fit_transform(features)

    if add_centroid:
        # Create a dataframe from the reduced features
        reduced_features_df = pd.DataFrame(reduced_features, 
                                        index=job_ids, 
                                        columns=[f"component_{i}" for i in range(1, n_components + 1)])
        # Add a reduced_centroid to the dataframe
        numeric_cols = reduced_features_df.columns
        reduced_centroid_features = reduced_features_df.mean(axis=0)
        reduced_centroid_df = pd.DataFrame(reduced_centroid_features).T  # Transpose to get it as a single row
        reduced_centroid_df.index = ["reduced_centroid"]  # Set index value to 'reduced_centroid'
        
        # Append the reduced_centroid_df to the main dataframe
        reduced_features_df = pd.concat([reduced_features_df, reduced_centroid_df])
        # Update the reduced_features and job_ids variables
        reduced_features = reduced_features_df.values
        job_ids = reduced_features_df.index.values


    # if add_centroid:
    #     reduced_features_df = pd.DataFrame(reduced_features, 
    #                                        index=job_ids, 
    #                                        columns=[f"component_{i}" for i in range(1, n_components + 1)])
    #     print(reduced_features_df)
    #     reduced_features_df = add_centroid_to_features(reduced_features_df,
    #                                                    job_id="reduced_centroid")
    #     print(reduced_features_df)
    #     reduced_features = reduced_features_df.to_numpy()
    #     job_ids = reduced_features_df.index.values

    # Calculate the variance explained by each principal component (if applicable)
    explained_variance = np.sum(reducer.explained_variance_ratio_) if dim_reduction == 'PCA' else None

    # Create a Plotly figure
    fig = go.Figure()

    # Add each job's features to the figure as a scatter plot
    for i, (job_id, feature) in enumerate(zip(job_ids, reduced_features)):
        color = 'blue'
        size = 6
        if job_id == central_job_id:  # Highlight the central job
            color = 'red'
            size = 8
        elif job_id in ["centroid", "reduced_centroid"]:  # Highlight the centroid
            color = 'gray'
            size = 8
        if n_components == 3:
            fig.add_trace(go.Scatter3d(
                x=[feature[0]],
                y=[feature[1]],
                z=[feature[2]],
                mode='markers+text',
                marker=dict(
                    size=size,
                    color=color,
                ),
                text=[str(job_id)[:5]],
                name=str(job_id)[:5]
            ))
        elif n_components == 2:
            fig.add_trace(go.Scatter(
                x=[feature[0]],
                y=[feature[1]],
                mode='markers+text',
                marker=dict(
                    size=size,
                    color=color,
                ),
                text=[str(job_id)[:5]],
                name=str(job_id)[:5]
            ))
        else:
            raise ValueError("Invalid number of components. n_components must be 2 or 3.")

    # Add the variance explained by each principal component to the figure's title (if applicable)
    title = f"{n_components}D plot of features ({dim_reduction}~: {explained_variance})" if explained_variance else f"{n_components}D plot of features ({dim_reduction})"
    fig.update_layout(title=title)

    # Return the figure
    return fig




def display_timeseries(workflows):
    """
    Display the time series of bytesRead and bytesWritten for a list of workflows using Plotly.
    Args:
        workflows (list of dict): A list of workflows. Each workflow is a dictionary 
                                  with 'bytesRead', 'bytesWritten', and 'timestamp'.
    """
    max_columns = 3
    n = len(workflows)
    n_cols = min(n, max_columns)
    n_rows = math.ceil(n / n_cols)

    fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=[list(w.keys())[0] for w in workflows])

    for i, workflow_data in enumerate(workflows):
        workflow_id = list(workflow_data.keys())[0]
        job = workflow_data[workflow_id]

        row = i // n_cols + 1
        col = i % n_cols + 1

        fig.add_trace(go.Scatter(
            x=job['timestamp'],
            y=job['bytesRead'],
            mode='lines',
            name='bytesRead',
            line_color='blue'
        ), row=row, col=col)

        fig.add_trace(go.Scatter(
            x=job['timestamp'],
            y=job['bytesWritten'],
            mode='lines',
            name='bytesWritten',
            line_color='orange'
        ), row=row, col=col)

    fig.update_layout(height=400*n_rows, width=400*n_cols, title_text="Time series for Jobs", showlegend=False)
    return fig




In [75]:
fig = display_features(central_job._features, dim_reduction="PCA", 
                       central_job_id=central_job_id,
                       n_components=2)
fig.show()


In [76]:
[list(workflow.keys())[0] for workflow in workflows]

['642b5fd8547fb888bac488ed',
 '633ab5ebac213a8b8529d186',
 '634019e6ac213a8b852f5f9c',
 '63402075ac213a8b852f6821',
 '634015a2ac213a8b852f4ec5',
 '63e52bfdac213a8b85e98c21',
 '63be91efac213a8b85bc56eb',
 '63be952fac213a8b85bc65c8',
 '63455953ac213a8b853535f4',
 '63bfc87bac213a8b85be494b',
 '643776f0547fb888bad22e64',
 '634d3df5ac213a8b853d659e',
 '633313fcac213a8b852232db']

In [77]:
display_timeseries(workflows)

In [13]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
import plotly.graph_objects as go
import numpy as np

# Apply PCA to reduce the features to 3 dimensions
pca = PCA(n_components=3)
reduced_features = pca.fit_transform(central_job.features)

# Calculate the amount of variance captured by each principal component
explained_variance = pca.explained_variance_ratio_

# Calculate the total amount of variance captured
total_variance = np.sum(explained_variance)

print(f"Variance captured by each principal component: {explained_variance}")
print(f"Total variance captured: {total_variance}")
# Calculate the Euclidean distance from each job to the centroid
distances = euclidean_distances(reduced_features, [centroid])

# Find the index of the job that is closest to the centroid
closest_job_idx = np.argmin(distances)

# Create the figure
fig = go.Figure()

# For each set of reduced features, create a scatter3d trace and add it to the figure
for i, feature in enumerate(reduced_features):
    # Choose different color and size for the closest job
    if i == closest_job_idx:
        color = 'red'
        size = 15  # Bigger size for the closest job
        name = f'Job {i} (Closest to Centroid)'
    else:
        color = i
        size = 12  # Normal size for other jobs
        name = f'Job {i}'

    fig.add_trace(go.Scatter3d(
        x=[feature[0]],
        y=[feature[1]],
        z=[feature[2]],
        mode='markers',
        marker=dict(
            size=size,
            color=color,                
            colorscale='Viridis',   
            opacity=0.8,
        ),
        text=[f'Job {i}'],
        name=name
    ))

# Add the centroid to the plot
fig.add_trace(go.Scatter3d(
    x=[centroid[0]],
    y=[centroid[1]],
    z=[centroid[2]],
    mode='markers',
    marker=dict(
        size=12,
        color='black',   # set color to black
        opacity=0.8,
    ),
    text=['Centroid'],
    name='Centroid'
))

# Set the layout properties
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='PC1'),
        yaxis=dict(title='PC2'),
        zaxis=dict(title='PC3')
    )
)

# Show the plot
fig.show()
